In [ ]:
import pandas as pd
import numpy as np 

pd.options.display.max_rows = 100
pd.options.display.max_columns = 50

from clean_data_functions import *

import warnings
warnings.filterwarnings("ignore")

In [ ]:
common_columns =[
 'Date',
 'Time',
 'Cool D-D',
 'Hi Temp',
 'In Air Density',
 'In Heat',
 'In Hum',
 'Low Temp',
 'Rain',
 'Soil Moist',
 'Soil Temp',
 'Temp In',
 'Temp Out',
 'Wind Run',
 'Wind Speed'
 ]

In [ ]:
df_merged = pd.read_excel("..\dados\clean\BSM_merged.xlsx")

In [ ]:
df_merged.info()

In [ ]:
df_final = df_merged[common_columns]
df_final

In [ ]:
df_final['Wind Speed'].replace('3 .6', '3.6', inplace=True)

In [ ]:
df_final['Wind Speed'] = df_final['Wind Speed'].astype('float16')

In [ ]:
df_final['Wind Speed'].unique()

## Análise de dados faltantes

In [ ]:
df_final['YEAR'] = df_final['Date'].dt.year
df_final['MONTH'] = df_final['Date'].dt.month

df_final.columns

In [ ]:
import matplotlib.pyplot as plt

cols = [col for col in common_columns if col not in ['Time', 'Date']]

non_null_counts = df_final.shape[0] - df_final[cols].isna().sum()

plt.figure(figsize=(15, 12))
bars = plt.bar(non_null_counts.index, non_null_counts.values)
plt.ylim(0, 62000)  
plt.title('BSM - Total de Valores Não Nulos por Coluna (Absoluto)')
plt.ylabel('Quantidade de Valores Não Nulos')
plt.xlabel('Variáveis')
plt.xticks(rotation=45)
# plt.tight_layout()
plt.grid(axis='y')

for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2.0, yval + 500, f'{int(yval)}\n({100 * int(yval)/df_final.shape[0]:.2f}%)', ha='center', va='bottom', fontsize=9)

plt.show()

In [ ]:
colunas_para_analisar = [col for col in common_columns if col not in ['Time', 'Date']]

grupo = df_final.groupby(['YEAR'])

non_null_by_month = grupo[colunas_para_analisar].count().reset_index()

for coluna in colunas_para_analisar:
    x = non_null_by_month['YEAR'].astype(str)
    y = non_null_by_month[coluna]

    plt.figure(figsize=(10, 5))
    plt.plot(x, y, marker='o', label=coluna)
    for xi, yi in zip(x, y):
        plt.text(xi, yi + 200, str(yi), ha='center', va='bottom', fontsize=8)

    plt.title(f'Total de Valores Não Nulos por Ano - {coluna}')
    plt.xlabel('Ano')
    plt.ylabel('Contagem de Valores Não Nulos')
    plt.xticks(rotation=45)
    plt.ylim(0, df_final[df_final['YEAR'] == 2020].shape[0]) 
    plt.grid(True)
    plt.tight_layout()
    plt.legend()
    plt.show()

In [ ]:
anos = sorted(df_final['YEAR'].unique())

# Preparar figura
n_rows = len(colunas_para_analisar)
n_cols = len(anos) + 1
fig_width = n_cols * 5
fig_height = n_rows * 4

fig, axes = plt.subplots(n_rows, n_cols, figsize=(fig_width, fig_height), sharey='row')

if n_rows == 1:
    axes = [axes]
if n_cols == 1:
    axes = [[ax] for ax in axes]

# Dados consolidados por ano
grupo = df_final.groupby(['YEAR'])
non_null_by_year = grupo[colunas_para_analisar].count().reset_index()
total_por_ano = df_final.groupby('YEAR').size()

# Preencher os gráficos
for row_idx, coluna in enumerate(colunas_para_analisar):
    for col_idx, ano in enumerate(anos):
        ax = axes[row_idx][col_idx]
        df_aux_ano = df_final[df_final['YEAR'] == ano]

        total_por_mes = df_aux_ano.groupby('MONTH')[colunas_para_analisar].size()
        nao_faltantes_por_mes = (
            df_aux_ano.groupby('MONTH')[colunas_para_analisar]
            .apply(lambda x: x.notnull().sum())
        )
        nao_faltantes_pct = nao_faltantes_por_mes.div(total_por_mes, axis=0) * 100

        valores = nao_faltantes_pct[coluna]
        absolutos = nao_faltantes_por_mes[coluna]
        ax.bar(valores.index, valores, color='skyblue')

        for mes, pct, abs_val in zip(valores.index, valores, absolutos):
            ax.text(mes, pct + 1, f'{pct:.1f}%\n({abs_val})', ha='center', va='bottom', fontsize=8)

        ax.set_title(f'{coluna} - {ano}')
        ax.set_xlabel('Mês')
        if col_idx == 0:
            ax.set_ylabel('% Não Nulos')
        ax.set_xticks(range(1, 13))
        ax.set_ylim(0, 110)
        ax.grid(axis='y', linestyle='--', alpha=0.7)

    # Gráfico de linha consolidado
    ax_line = axes[row_idx][-1]
    x = non_null_by_year['YEAR'].astype(str)
    y_abs = non_null_by_year[coluna]
    y_pct = (y_abs / total_por_ano.values) * 100
    ax_line.plot(x, y_pct, marker='o', label=coluna, color='orange')

    for xi, yi_pct, yi_abs in zip(x, y_pct, y_abs):
        ax_line.text(xi, yi_pct + 1, f'{yi_pct:.1f}%\n({yi_abs})', ha='center', va='bottom', fontsize=8)

    ax_line.set_title(f'{coluna} - Total Anual')
    ax_line.set_xlabel('Ano')
    ax_line.set_ylabel('% Não Nulos')
    ax_line.set_ylim(0, 110)
    ax_line.grid(True)
    ax_line.legend()

plt.tight_layout()
plt.show()


## Análise de correlação

In [ ]:
df_final

In [ ]:
df_final[['Hi Temp', 'Low Temp', 'Temp Out']].head(10)

In [ ]:
import seaborn as sns

# Calcular correlação

columns_to_corr = [c for c in df_final.columns if c not in ['YEAR', 'MONTH']]
corr = df_final[columns_to_corr].corr()

# Plotar heatmap
plt.figure(figsize=(14, 10))
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5, square=True)
plt.title('BSM - Correlação entre Variáveis Numéricas')
plt.tight_layout()
plt.show()

## Análise de Série Temporal

In [ ]:
# Reimportar bibliotecas após reset e simular df_final novamente
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np

In [ ]:
df_final_plot = df_final.copy(deep=True)

df_final_plot['Date'] = pd.to_datetime(df_final_plot['Date'])
# df_final_plot.set_index('Date', inplace=True)

# Lista de colunas numéricas para plotar
colunas_para_plotar = [c for c in common_columns if c not in ['Date', 'Time']]
colunas_para_plotar = [colunas_para_plotar[12]]

df_final_plot[colunas_para_plotar] = df_final_plot[colunas_para_plotar].where(
    #(df_final_plot[colunas_para_plotar] > 1) & (df_final_plot[colunas_para_plotar] <= 180),
    ( df_final_plot[colunas_para_plotar] <= 10),
    np.nan
)

# Criar gráficos de série temporal por variável (cada uma em uma figura)
for col in colunas_para_plotar:
    plt.figure(figsize=(14, 4))
    plt.plot(df_final_plot['Date'], df_final_plot[col], label=col, linewidth=1)
    plt.title(f'Série Temporal - {col} (2017 a 2023)')
    plt.ylabel(col)
    plt.xlabel('Ano')
    plt.grid(True)
    plt.legend()
    plt.axvline(pd.Timestamp('2018-01-01'), color='gray', linestyle='--', linewidth=0.7)
    plt.axvline(pd.Timestamp('2019-01-01'), color='gray', linestyle='--', linewidth=0.7)
    plt.axvline(pd.Timestamp('2020-01-01'), color='gray', linestyle='--', linewidth=0.7)
    plt.axvline(pd.Timestamp('2021-01-01'), color='gray', linestyle='--', linewidth=0.7)
    plt.axvline(pd.Timestamp('2022-01-01'), color='gray', linestyle='--', linewidth=0.7)
    plt.axvline(pd.Timestamp('2023-01-01'), color='gray', linestyle='--', linewidth=0.7)
    plt.gca().xaxis.set_major_locator(mdates.YearLocator())
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
    plt.tight_layout()
    plt.show()

In [ ]:
# df_final_plot['Low Temp'].hist()

In [ ]:
var = colunas_para_plotar[0]


print(df_final_plot[var].describe())
print(var)
plt.title(f"Distribuição {var}")
df_final_plot[var].plot(kind='hist')
plt.show()

In [ ]:
df_final_aux = df_final[(df_final[var] > 10)]
# vars_to_show = ['Date']
# vars_to_show.append(var)
# df_final_aux[vars_to_show]
df_final_aux

In [ ]:
# Barra de São Miguel - Município